# Natural Language Processing with RNNs & Attention

When Alan Turing  imagined his famous Turing test in 1950, his objective was to evaluate a machine's ability to match human intelligence. He could have tested for many things, such as the ability to recognise cats in pictures, play chess, compose music, or escapea maze, but interestingly, he chose a linguistic task. More specifically, he devised a *chatbot* capable of fooling its interlocutor into thinking it was human. This test does have its weaknesses: a set of hardcoded rules can fool unsuspecting or naive humans (e.g., the machine could give vague predefined answers in response to some keywords; it could pretend that it is joking or drunk, to get a pass on its weirdest answers; or it could escape difficult questions by answering them with its otwn questions), & many aspects of human intelligence are utterly ignored (e.g., the ability to interpret nonverbal communication such as facial expressions, or to learn a manual task). But the test does highlight the fact that mastering language is arguably *Homo sapien*'s greated cognitive ability. Can we build a machine that can read & write natural language?

A common approach for natural language tasks is to use recurrent neural networks. We will therefore continue to explore RNNs, starting with a *character* RNN, trained to predict the next character in a sentence. This will allow us to generate some original text, & in the process, we will see how to build a tensorflow dataset on a very long sequence. We will first use a *stateless* RNN (which learns on random portions of text at each iteration, without any information on the rest of the text), then we will build a *stateful* RNN (which preserves the hidden state between training iterations & continues reading where it left off, allowing it to learn longer patterns). Next, we will build a RNN to perform sentiment analysis (e.g., reading movie reviews & extracting the rater's feelings about the movie), this time treating sentences as sequences of words, rather than characters. Then we will show how RNNs can be used to build an encoder-decoder architecture capable of performing neural machine translation (NMT). For this, we will use the seq2seq API provided by the TensorFlow addons project.

In the second part of this lesson, we will look at *attention* mechanisms. As their name suggests, these are neural network components that learn to select the part of the inputs that the rest of the model should focus on at each time step. First, we will see how to boost the performance of an RNN-based encoder-decoder architecture using attention, then we will drop RNNs altogether & look at very successful attention-only architecture called the *transformer*. Finally, we will take a look at some of the most important advances in NLP, including incredibly powerful language models such as GPT-2 & BERT, both based on transformers.

We'll start with a simple & fun model that can write like Shakespeare (sort of).

---

# Generating Shakespearean Text Using a Character RNN

In a famous 2015 blog post titled "The Unreasonable Effectiveness of Recurrent Neural Networks", Andrej Karpathy showed how to train an RNN to predict the next character in a sentence. This *Char-RNN* can then be used to generate novel text, one character at a time. Here is a small sample of the text generated by a Char-RNN model after it was trained on all of Shakespeare's work:

    PANDARUS:
    Alas, I think he shall be come approached & the day
    When litle srain would be attain'd into being never fed,
    & who is but a chain & subjects of his death,
    I should not sleep.

Not exactly a masterpice, but it is still impressive that the model was able to learn words, grammar, proper punctuation, & more, just by learning to predict the next character in a sentence. Let's look at how to build a Char-RNN, step by step, starting with the creation of the dataset.

## Creating the Training Dataset

First, let's download all of Shakespeare's work, using kera's handy `get_file()` function & downloading the data from Andrej Karpath's char-RNN project:

In [ ]:
import tensorflow as tf
from tensorflow import keras

shakespeare_url = "https://homl.info/shakespeare"
file_path = keras.utils.get_file("shakespeare.txt", shakespeare_url)
with open(file_path) as f:
    shakespeare_text = f.read()

Next, we must encode every character as an integer. One option is to create a custom preprocessing layer. But in this case, it will be simpler to use kera's `Tokenizer` class. First, we need to fit a tokenizer to the text: it will find all the characters used in the text & map each of them to a different character ID, from 1 to the number of distinct characters (it does not start at 0, so we can use that value for masking):

In [ ]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level = True)
tokenizer.fit_on_texts(shakespeare_text)

We set `char_level = True` to get character-level encoding rather than the default word-level encoding. Note that this tokenizer converts the text to lower case by default (but you can set `lower = False` if you do not want that). Now the tokenizer can encode a sentence (or a list of sentences) to a list of character IDs & back, & it tells us how many distinct characters there are & the total number of characters in the text:

In [ ]:
tokenizer.texts_to_sequences(["First"])

In [ ]:
tokenizer.sequences_to_texts([[20, 6, 9, 8, 3]])

In [ ]:
max_id = len(tokenizer.word_index) # number of distinct characters
dataset_size = tokenizer.document_count # total number of characters

Let's encode the full text so each character is represented by its ID (we subtract 1 to get IDs from 0 to 38, rather than from 1 to 39):

In [ ]:
import numpy as np

[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1

Before we continue, we need to split the dataset into a training set, a validation set, & a test set. We can't just shuffle all the characters in the text, so how do you split a sequential dataset?

## How to Split a Sequential Dataset

It is very important to avoid any overlap between the training set, the validation set, & the test set. For example, we can take the first 90% of the text for the training set, then the next 5% for the validation set, & the final 5% for the text set. It would also be a good idea to leave a gap between these sets to avoid the risk of a paragraph overlapping over two sets.

When dealing with time series, you would in general split across time: for example, you might take years 2000 to 2012 for the training set, the years 2013 to 2015 for the validation set, & the years 2016 to 2018 for the test set. However, in some cases, you may be able to split along other dimensions, which will give you a longer time period to train on. For example, if you have data about the financial health of 10,000 companies from 2000 to 2018, you might be able to split this data across the different companies. It's very likely that many of these companies will be strongly correlated, though (e.g., whole economic sectors may go up or down jointly), & if you have correlated companies across the training set & the test set, your test set will not be as useful, as its measure of the generalisation error will be optimistically biased.

So, it is often safer to split across time -- but this implicitly assumes that the patterns the RNN can learn in the past (in the training set) will still exist in the future. In other words, we assume that the time series is *stationary* (at least in a wide sense). For many time series, this assumption is reasonable (e.g., chemical reactions should be fine, since the laws of chemistry don't change every day), but for many other it is not (e.g., financial markets are notoriously not stationary since patterns disappear as soon as traders spot them & start exploiting them). To make sure the time series is indeed sufficiently stationary, you can plot the model's errors on the validation set across time: if the model performs much better on the first part of the validation set than on the last part, then the time series may not be stationary enough, & you might be better off training the model on a shorter time span.

In short, splitting a time series into a training set, a validation set, & a test set is not a trivial task, & how it's done will depend strongly on the task at hand.

Now back to Shakespeare. Let's take the first 90% of the text for the training set (keeping the rest for the validation set & the test set), & create a `tf.data.Dataset` that will return each character one by one from this set:

In [ ]:
train_size = dataset_size * 90  // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

## Chopping the Sequential Dataset into Multiple Windows

The training set now consists of a single sequence of over a million characters, so we can't just train the neural network directly on it: the RNN would be equivalent to a deep net with over a million layers & we would have a single (very long) instance to train it. Instead, we will use the dataset's `window()` method to convert this long sequence of characters into many smaller windows of text. Ever instance in the dataset will be a fairly short substrings of the whole text, & the RNN will be unrolled only over the length of these substrings. This is called *truncated backpropagation through time*. Let's call the `window()` method to create a dataset of short text windows.

In [ ]:
n_steps = 100
window_length = n_steps + 1
dataset = dataset.window(window_length, shift = 1, drop_remainder = True)

By default, the `window()` method creates nonoverlapping windows, but to get the largest possible training set we use `shift = 1` so that the first window contains characters 0 to 100, the second contains characters 1 to 101, & so on. To ensure that all windows are exactly 101 characters long (which will allow us to create batches without having to do any padding), we set `drop_remainder = True` (otherwise the last 100 windows will contain 100 chracters, 99 chracters, & so on down to 1 character).

The `window()` method creates a dataset that contains windows, each of which is also represented as a dataset. It's a *nested dataset* analogous to a list of lists. This is useful when you want to transform each window by calling its dataset methods (e.g., to shuffle them or batch them). However, we cannot use a nested dataset directly for training, as our model will expect tensors as input, not datasets. So we must call the `flat_map()` method: it converts a nested dataset into a *flat dataset* (one that does not contain datasets). For example, suppose {1, 2, 3} represented a dataset containing the sequence of tensors 1, 2, & 3. If you flatten the nested dataset {{1, 2}, {3, 4, 5, 6}}, you will get back the flat dataset {1, 2, 3, 4, 5, 6}. Moreover, the `flat_map()` method takes a function as an argument, which allows you to transform each dataset in the nested dataset before flattening. For example, if you pass the function `lambda ds: ds.batch(2)` to `flat_map()`, then it will transform the nested dataset {{1, 2}, {3, 4, 5, 6}} into the flat dataset {[1, 2], [3, 4], [5, 6]}: it's a dataset of tensors of size 2. With that in mind, we are ready to flatten our dataset:

In [ ]:
dataset = dataset.flat_map(lambda window: window.batch(window_length))

Notice that we call `batch(window_length)` on each window: since all windows have exactly that length, we will get a single tensor for each of them. Now the dataset contains consevutive windows fo 101 characters each. Since gradient descent works best when the instances in the training set are independent & identically distributed, we need to shuffle these windows. Then we can batch the windows & separate the inputs (the first 100 characters) from the target (the last character):

In [ ]:
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

The below figure summarises the dataset preparation steps so far (showing windows of length 11 rather than 101, & a batch size of 3 instead of 32).

<img src = "Images/Shuffled Windows.png" width = "600" style = "margin:auto"/>

As discussed in previous lessons, categorical input feature should generally be encoded, usually as one-hot vectors or as embeddings. Here, we will encode each characterusing a one-hot vector because there are fairly few distinct characters (only 39):

In [ ]:
dataset = dataset.map(lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth = max_id), Y_batch))

Finally, we just need to add prefetching:

In [ ]:
dataset = dataset.prefetch(1)

That's it! Preparing the dataset was the hardest part. Now let's create the model.

## Building & Training the Char-RNN Model

To predict the next character based on the previous 100 characters, we can use a RNN with 2 `GRU` layers ofr 128 units each & 20% dropout on both the inputs (`dropout`) & the hidden states (`recurrent_dropout`). We can tweak these hyperparameters layer, if needed. The output layer is a time-distributed `Dense` layer that must have 39 units (`max_id`) because there are 39 distinct characters in the text, & we want to output a probability for each possible character (at each time step). The output probabilities should sum up to 1 at each time step, so we apply the softmax activation function to the outputs of the `Dense` layer. We can then compile this model, using the `"sparse_categorical_crossentropy"` loss & an adam optimiser. Finally, we are ready to train the model for several epochs.

In [ ]:
model = keras.models.Sequential([
    keras.layers.InputLayer(shape = [None, max_id]),
    keras.layers.GRU(128, return_sequences = True, dropout = 0.2),
    keras.layers.GRU(128, return_sequences = True, dropout = 0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id, activation = "softmax"))
])
model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam")
model.fit(dataset, epochs = 10)

## Using the Char-RNN Model

Now we have a model that can predict the next character in text written by Shakespeare. To feed it some text, we first need to preprocess it like we did earlier, so let's create a little function for this:

In [ ]:
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

Now, let's use the model to predict the next letter in some text:

In [ ]:
X_new = preprocess(["How are yo"])
Y_pred = np.argmax(model(X_new), axis = -1)
tokenizer.sequences_to_texts(Y_pred + 1)[0][-1]

## Generating Fake Shakespearean Text

To generate new text using the Char-RNN model, we could feed it some text, make the model predict the most likely next letter, add it at the end of the text, then give the extended text to the model to guess the next letter, & so on. But in practice, this often leads to the same words being repeated over & over again. Instead, we can pick the next character randomly, with a probability equal to the estimated probability using TensorFlow's `tf.random.categorical()` function. This will generate more diverse & interesting text. The `categorical()` function samples random class indices, fiven the class log probabilities (logits). To have more control over the diversity of the generated text, we can divide the logits by a number called the *temperature*, which we can tweak as we wish: a temperature close to 0 will favour the high probability characters, while a very high temperature will give all characters an equal probability. The following `next_char()` function uses this approach to pixk the next character to add to the input text:

In [ ]:
def next_char(text, temperature = 1):
    X_new = preprocess([text])
    y_proba = model.predict(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples = 1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

Next, we can write a small function that will repeatedly call `next_char()` to get the next character & append it to the given text:

In [ ]:
def complete_text(text, n_chars = 50, temperature = 1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

We are not ready to generate some text! Let's try with different temperatures:

In [ ]:
print(complete_text("t", temperature = 0.2))

In [ ]:
print(complete_text("w", temperature = 1))

In [ ]:
print(complete_text("w", temperature = 2))

Apparently, our Shakespeare model works best at a temperature close to 1. To generate more convicing text, you could try using more `GRU` layers & more neurons per layer & train for longer, & add more regularisation by increasing dropout rate in the `GRU` layers. Moreover the model is currently incapable of learning patterns longer than `n_steps`, which is just 100 characters. You could try making this window larger, but it will also make training harder, & even LSTM & GRU cells cannot handle very long sequences. Alternatively, you could use a stateful RNN.

## Stateful RNN

Until now, we have used only *stateless RNNs*: at each training iteration, the model starts with a hidden state full of zeros, then it updates this state at each time step, & after the last time step, it throws it away, as it is not needed anymore. What if we told the RNN to preserve this final state after processing one training batch & use it as the initial state for the next training batch? This way the model can learn long-term patterns despite only backpropagating through short sequences. This is called a *stateful RNN*. Let's see how to build one.

First, note that a stateful RNN only makes sense if each input sequence in a batch starts exactly where the corresponding sequence in the previous batch left off. So the first thing we need to do to build a stateful RNN is to use sequential & nonoverlapping input sequences (rather than the shuffled & overlapping sequences we used to train stateless RNNs). When creating the `Dataset`, we must therefore use `shift = n_steps` (instead of `shift = 1`) when calling the `window()` method. Moreover, we must obviously *not* call the `shuffle()` method. Unfortunately, batching is much harder when preparing a dataset for a stateful RNN than it is for a stateless RNN. Indeed, if we were to call `batch(32)`, then 32 consecutive windows would be put in the same batch, & the following batch would not continue each of these window where it left off. The first batch would contain windows 1 to 32 & the second batch would contain windows 33 to 64, so if you consider, say, the first window of each batch (i.e., windows 1 & 33), you can see that they are not consecutive. The simplest solution to this problem is to just use "batches" containing a single window:

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])
dataset = dataset.window(window_length, shift = n_steps, drop_remainder = True)
dataset = dataset.flat_map(lambda window: window.batch(window_length))
dataset = dataset.batch(1)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth = max_id), Y_batch))
dataset = dataset.prefetch(1)

The figure below summarise the first steps:

<img src = "Images/Consecutive Sequence Fragments for Stateful RNNs.png" width = "600" style = "margin:auto"/>

Batching is harder, but it is not impossible. For example, we could chop Shakespeare's text into 32 texts of equal length, create one dataset of consecutive input sequences for each of them, & finally use `tf.train.Dataset.zip(datasets).map(lambda * windows: tf.stack(windows))` to create proper consecutive batches, where the $n^{th}$ input sequence in a batch starts off exactly where the $n^{th}$ input sequence in the previous batch.

Now let's create the stateful RNN. First, we need to set `stateful = True` when creating every recurrent layer. Second, the stateful RNN needs to know the batch size (since it will preserve a state for each input sequence in the batch). so we must set the `batch_input_shape` argument in the first layer. Note that we can have the second dimension unspecified, since the inputs could have any length:

In [ ]:
model = keras.models.Sequential([
    keras.layers.InputLayer(batch_input_shape = [batch_size, None, max_id]),
    keras.layers.GRU(128, return_sequences = True, stateful = True,
                     dropout = 0.2),
    keras.layers.GRU(128, return_sequences = True, stateful = True,
                     dropout = 0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id, activation = "softmax"))
])

At the end of each epoch, we need to reset the states before we go back to the beginning of the text. For this, we can use a small callback:

In [ ]:
class ResetStatesCallback(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

Now, we can compile & fit the model (for more epochs, because each epoch is shorter than earlier & there is only one instance per batch):

In [ ]:
model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam")
model.fit(dataset, epochs = 40, callbacks = [ResetStatesCallback])

Now that we have built a character-level model, it's time to look at word-level models & tackle a common natural language processing task: *sentiment analysis*. In the process, we will learn how to handle sequences of variable lengths using masking.

---

# Sentiment Analysis

If MNIST is the "hello world" of computer vision, then the IMDb reviews dataset is the "hello world" of natural language processing: it consists of 50,000 movie reviews in English (25,000 for training, 25,000 for testing) extracted from the famous internet movie database, along with a simple binary target for each review indicating whether it is negative (0) or positive (1). Just like MNIST, the IMDb reviews dataset is popular for good reasons: it is simple enough to be tackled on a laptop in a reasonable amount of time, but challenging enough to be fun & rewarding. Keras provides a simple function to load it:

In [ ]:
(X_train, y_train), (X_test, y_test) = keras.datasets.imdb.load_data()
X_train[0][:10]

Where are the movie reviews? Well, as you can see, the dataset is already preprocessed for you: `X_train` consists of a list of reviews, each of which is represented as a numpy array of integers, where each integer represents a word. All punctuation was removed, & then words were converted to lowercase, split by spaces, & finally indexed by frequences (so low integers correspond to frequent words). The integers, 0, 1, & 2 are special: they represent the padding token, the *start-of-sequence* (SSS) token, & unknown words, respectively. If you want to visualise a review, you can decode it like this:

In [ ]:
word_index = keras.datasets.imdb.get_word_index()
id_to_word = {id_ + 3: word for word, id_ in word_index.items()}
for id_, token in enumerate(("<pad>", "<sos>", "<unk>")):
    id_to_word[id_] = token

" ".join([id_to_word[id_] for id_ in X_train[0][:10]])

In a real project, you will have to preprocess the text yourself. You can do that using the same `Tokenizer` class we used earlier, but this time setting `char_level = False` (which is the default). When encoding words, it filters out a lot of characters, including most punctuation, line breaks, & tabs (but you can change this by setting the `filters` argument). Most importantly, it uses spaces to identify word boundaries. This is ok for English & many other scripts (written languages) that use spaces between words, but not all scripts use spaces this way. Chinese does not use spaces between words, Vietnamese uses spaces even within words, & languages such as German often attach multiple words together, without spaces. Even in English, spaces are not always the best way to tokenize text: think of "San Francisco" or "#ILoveDeepLearning".

Fortunately, there are better options. The 2018 paper by Taku Kudo introduced an unsupervised learning technique to tokenise & detokenise text at the subword level in a language-independent way, treating spaces like other characters. With this approach, even if your model encounters a word it has never seen before, it can still reasonably guess what it means. For example, it may never have seen the word "smartest" during training, but perhaps it learned the word "smart" & it also learned that the suffix "est" means "the most", so it can infer the meaning of "smartest". Google's sentencepiece project provides an open source implementation, described in a paper by Taku Kudo & John Richardson.

Another option was proposed in an earlier paper by Rico Sennrich et al. that explored other ways of creating subword encodings (e.g., using *byte pairing encoding*). Last but not least, the TensorFlow team released the tf.text library in June 2019, which implements various tokenization strategies, including wordpiece (a variant of byte pair encoding).

If you want to deploy your model to a mobile device or a web browser, & you don't want to have to write a different preprocessing function every time, then you will want to handle preprocessing using only TensorFlow operations, so it can be included in the model itself. Let's see how. First, let's load the original IMDb reviews, as text (byte strings), using TensorFlow datasets:

In [ ]:
import tensorflow_datasets as tfds

datasets, info = tfds.load("imdb_reviews", as_supervised = True, with_info = True)
train_size = info.splits["train"].num_examples

Next, let' write the preprocessing function:

In [ ]:
def preprocess(X_batch, y_batch):
    X_batch = tf.strings.substr(X_batch, 0, 300)
    X_batch = tf.strings.regex_replace(X_batch, b"<br\\s*/?>", b" ")
    X_batch = tf.strings.regex_replace(X_batcj, b"[^a-zA-A']", b" ")
    x_batch = tf.strings.split(X_batch)
    return X_batch.to_tensor(default_value = b"<pad>"), y_batch

It starts by truncating the reviews, keeping only the first 300 characters of each: this will speed up training, & it won't impact performance too much because you can generally tell whether a review is positive or not in the first sentence or two. Then it uses *regular expressions* to replace `<br />` tags with spaces, & to replace any characters other than letters & quotes with spaces. For example, the text `"Well, I can't <br />` will become `Well I can't"`. Finally, the `preprocess()` function splits the reviews by the spaces, which returns a ragged tensor, & it converts this ragged tensor to a dense tensor, padding all reviews with the padding token `"<pad>"` so that they all have the same length.

Next, we need to construct the vocabulary. This requires going through the whole training set once, applying our `preprocess()` function, & using a `Counter` to count the number of occurences of each word:

In [ ]:
from collections import Counter

vocabulary = Counter()
for X_batch, y_batch in datasets["train"].batch(32).map(preprocess):
    for review in X_batch:
        vocabulary.update(list(review.numpy()))

Let's look at the three most common words:

In [ ]:
vocabulary.most_common()[:3]

Great! We probably don't need our model to know all the words in the dictionary to get good performance, though, so let's truncate the vocabulary, keeping only the 10,000 most common words:

In [ ]:
vocab_size = 10000
truncated_vocabulary = [word for word, count in vocabulary.most_common()[:vocab_size]]

Now we need to add a preprocessing step to replace each word with its ID (i.e., its index in the vocabulary). Just like we did before, we will create a lookup table for this, using 1000 out-of-vocabulary (oov) buckets:

In [ ]:
words = tf.constant(truncated_vocabulary)
word_ids = tf.range(len(truncated_vocabulary), dtype = tf.int64)
vocab_init = tf.lookup.KeyValueTensorInitializer(words, word_ids)
num_oov_buckets = 1000
table = tf.lookup.StaticVocabularyTable(vocab_init, num_oov_buckets)

We can then use this table to look up the IDs of a few words:

In [ ]:
table.lookup(tf.constant([b"This movie was faaaaaantastic".split()]))

Note that the words "this", "movie", & "was" were found in the table, so their IDs are lower than 10,000, while the word "faaaaaaantastic" was not found, so it was mapped to one of the oov buckets, with an ID greater than or equal to 10,000.

Now we are ready to create the final training set. We batch the reviews, then convert them to short sequences of words using the `preprocess()` function, then encode these words using a simple `encode_words()` function that uses the table we just built & finally prefetch the next batch:

In [ ]:
def encode_words(X_batch, y_batch):
    return table.lookup(X_batch), y_batch

train_set = datasets["train"].batch(32).map(preprocess)
train_set = train_set.map(encode_words).prefetch(1)

At last, we can create the model & train it:

In [ ]:
embed_size = 128
model = keras.models.Sequential([
    keras.layers.InputLayer(shape = [None]),
    keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size),
    keras.layers.GRU(128, return_sequences = True),
    keras.layers.GRU(128),
    keras.layers.Dense(1, activation = "sigmoid")
])
model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])
model.fit(train_set, epochs = 5)

The first layer is an `Embedding` layer, which will convert word IDs into embeddings. The embedding matrix needs to have one row per word ID (`vocab_size` + `num_oov_buckets`) & one column per embedding dimension (this example uses 128 dimensions, but this is a hyperparameter you can tune). Whereas the inputs of the model will be 2D tensors of shape [*batch size*, *time steps*], the output of the `Embedding` layer will be a 3D tensor of shape [*batch size*, *time steps*, *embedding size*].

The rest of the model is fairly straightforward: it is composed of two `GRU` layers, with the second one returning only the output of the last time step. The output layer is just a single neuron using the sigmoid activation function to output the estimated probability that the review expresses a positive sentiment regarding the movie. We then compile the model quite simply, & we fit it on the dataset we prepared earlier, for a few epochs.

## Masking

As it stands, the model will need to learn that the padding tokens should be ignored. But we already know that! Why don't we tell the model toignore the padding tokens, so that it can focus on the data that actually matters? It's actually trivial: simply add `mask_zero = True` when creating the `Embedding` layer creates a *mask tensor* equal to `K.not_equal(inputs, 0)` (where `K = keras.backend`): it is a boolean tensor with the same shape as the inputs, & it is equal to `False` anywhere the word IDs are 0, or `True` otherwise. This mask tensor is then automatically propagated by the model to all subsequent layers, as long as the time dimension is preserved. So in this example, both `GRU` layers will receive this mask automatically, but since the second `GRU` layer does not return sequences (it only returns the output of the last time step), the mask will not be transmitted to the `Dense` layer. Each layer may handle the mask differently, but in general, they simply ignore masked time steps (i.e., time steps for which the mask is `False`). For example, when a recurrent layer encounters a masked time step, it simply copies the output from the previous time step. If the mask propagates all the way to the output (in models that output sequences, which is not the case in this example), then it will be applied to the losses as well, so the masked time steps will not contribute to the loss (their loss will be 0).

All layers that receive the mask must support masking (or else an exception will be raised). This includes all recurrent layers, as well as the `TimeDistributed` layer & a few other layers. Any layer that supports masking must have a `supports_masking` attribute equal to `True`. If you want to implement your own custom layer with masking support, you should add a `mask` argument to the `call()` method (& obviously make the method use the mask somehow). Additionally, you should set `self.supports_masking = True` in the constructor. If your layer does not start with an `Embedding` layer, you may use the `keras.layers.Masking` layer instead: it sets the mask to `K.any(K.not_equal(inputs, 0), axis = -1)`, meaning that time steps where the last dimension is full of zeros will be masked out in subsequent layers (again, as long as the time dimension exists).

Using masking layers & automatic mask propagation works best for simple `Sequential` models. It will not always work for more complex models, such as when you need to mix `Conv1D` layers with recurrent layers. In such cases, you will need to explicitly compute the mask & pass it to the appropriate layers, using either the functional API or the subclassing API. For example, the following model is identical to the previous model, except it is built using the functional API & handles masking manually:

In [ ]:
K = keras.backend
inputs = keras.layers.InputLayer(shape = [None])
mask = keras.layers.Lambda(lambda inputs: K.not_equal(inputs, 0))(inputs)
z = keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size)(inputs)
z = keras.layers.GRU(128, return_sequences = True)(z, mask = mask)
z = keras.layers.GRU(128)(z, mask = mask)
outputs = keras.layers.Dense(1, activation = "sigmoid")(z)
model = keras.Model(inputs = [inputs], outputs = [outputs])

After training for a few epochs, this model will become quite good at judging whether a review is positive or not. If you use the `TensorBoard()` callback, you can visualize the embeddings in tensorboard as they are being learned: it is fascinating to see words like "awesome" & "amazing" gradually cluster on one side of the embedding space, while words like "awful" & "terrible" cluster on the other side. Some words are not as positive as you might expect (at least with this model), such as the word "good", presumably because many negative reviews contain the phrase "not good". It's impressive that the model is able to learn useful word embeddings based on just 25,000 movie reviews. Imagine how good the embeddings would be if we had billions of reviews to train on! Unfortunately, we don't, but perhaps, we can reuse word embeddings trained on some other large text corpus (e.g., wikipedia articles) even if it is not composed of movie reviews? After all, the word "amazing" generally has the same meaning whether or not you use it to talk about movies or anything else. Moreover, perhaps embeddings would be useful for sentiment analysis even if they were trained on another task: since words like "awesome" & "amazing" have a similar meaning, they will likely cluster in the embedding space even for other tasks (e.g., predicting the next word in a sentence). If all positive words & all negative words form clusters, then this will be helpful for sentiment analysis. So instead of using so many parameters to learn word embeddings, let's see if we can't just reuse pretrained embeddings.

## Reusing Pretrained Embeddings

The tensorflow hub project makes it easy to reuse pretrained model components in your own models. These model components are called *modules*. Simply browse the tf hub repository, find the one you need, & copy the code example into your project, & the module will automatically downloaded, along with its pretrained weights, & included in your model. Easy!

For example, let's use the `nnlm-en-dim50` sentence embedding module, version 1, in our sentiment analysis model:

In [ ]:
import tensorflow_hub as hub

model = keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/tf2-preview/nnlm-en-dim50/1",
                   dtype = tf.string, input_shape = [], outputs_shape = [50]),
    keras.layers.Dense(128, activation = "relu"),
    keras.layers.Dense(1, activation = "sigmoid")
])
model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])

The `hub.KerasLayer` layer downloads the module from the given URL. This particular module is a *sentence encoder*: it takes strings as input & encodes each one as a single vector (in this case, a 50-dimensional vector). Internally, it parses the string (splitting the words on spaces) & embeds each word using an embedding matrix that was pretrained on a huge corpus: the google news 7b corpus (7 billion words long). Then it computes the mean of all the word embeddings, & the result is the sentence embedding. We can then add two simple `Dense` layers to create a good sentiment analysis model. By default, a `hub.KerasLayer` is not trainable, but you can set `trainable = True` when creating it to change that so you can fine-tune it for your task.

Next, we can just load the IMDb reviews dataset -- no need to preprocess it (except for batching & prefetching) -- & directly train the model:

In [ ]:
datasets, info = tfds.load("imdb_reviews", as_supervised = True, with_info = True)
train_size = info.splits["train"].num_examples
batch_size = 32
train_set = dataset["train"].batch(batch_size).prefetch(1)
model.fit(train_set, epochs = 5)

Note that the last part of the tf hub module URL specified that we wanted version 1 of the model. This versioning ensures that if a new module version is released, it will not break our model. Conveniently, if you just enter this URL in a web browser, you will get the documentation for this module. By default, tf hub will cache the downloaded files into a local system's temporary directory. You may prefer to download them into a more permanent director to avoid having to download them again after every system cleanup. To do that, set the `TFHUB_CACHE_DIR` environment variable to the directory of your choice (e.g., `os.environ["TFHUB_CACHE_DIR"] = "./my_tfhub_cache"`).

So far, we have looked at time series, text generation using char-RNN, & sentiment analysis using word-level RNN models, training our own word embeddings or reusing pretrained embeddings. Let's now look at another important NLP task: *neural machine translation* (NMT), first using a pure encoder-decoder model, then improving it with attention mechanisms, & finally looking at the extraordinary transformer architecture.

---

# An Encoder-Decoder Network for Neural Machine Translation

Let's take a look at a simple neural machine translation model that will translate english sentences to french.

<img src = "Images/Simple Machine Translation Model.png" width = "600" style = "margin:auto"/>

In short, the english sentences are fed to the encoder, & the decoder outputs the french translations. Note that the french translations are also used as inputs to the decoder, but shifted back by one step. In other words, the decoder is given as input the word that it *should* have output at the previous step (regardless of what it actually output). For the very first word, it is given the start-of-sequence (SOS) token. The decoder is expected to end the sentence with an end-of-sequence (EOS) token.

Note that the english sentences are reverse before they are fed to the encoder. For example, "I drink milk" is reversed to "milk drink I". This ensures that the beginning of the english sentences will be fed last to the encoder, which is useful because that's generally the first thing that the decoder needs to translate.

Each word is initially represented by its ID (e.g., 288 for the word "milk"). Next, an `embedding` layer returns the word embedding. These word embeddings are what is actually fed to the encoder & the decoder.

At each step, the decoder outputs a score for each word in the output vocabulary (i.e., French), & then the softmax layer turns these scores into probabilities. For example, at the first step, the word "Je" may have a probability of 20%, "Tu" may have a probability of 1%, & so on. The word with the highest probability is the output. This is very much like a regular classification task, so you can train the model using the `"sparse_categorical_crossentropy" loss, much like we did in the char-RNN model.

Note that at inference time (after training), you will not have the target sentence to feed to the decoder. Instead, simply feed the decoder the word that it output at the previous step (this will require an embedding lookup that is now shown in the below diagram).

<img src = "Images/Feeding Previous Output as Input at Inference Time.png" width = "500" style = "margin:auto"/>

Ok, now you have the big picture. Still, there are a few more details to handle if you implement this model:

* So far, we have assumed that all input sequences (to the encoder & to the decoder) have a constant length. But obviously, sentence lengths vary. Since regular tensors have fixed shapes, they can only contain sequences of the same length. You can use masking to handle this, as discussed earlier. However, if the sentences have very different lengths, you can't just crop them like we did for sentiment analysis (because we want full translations, not cropped translations). Instead, group sentences into buckets of similar lengths (e.g., a bucket for the 1- to 6-word sentences, another for the 7- to 12- word sentences, & so on), using padding for the shorter sequences to ensure all sentences in a bucket have the same length (check out the `tf.data.experimental.bucket_by_sequence_length()` function for this). For example, "I drink milk" becomes "\<pad> \<pad> \<pad> milk drink I".
* We want to ignore any output past the EOS token, so these tokens should not contribute to the loss (they must be masked out). For example, if the model outputs "Je bois du lait \<eos> oui", the loss for the last word should be ignored.
* When the output vocabulary is large (which is the case here), outputting a probability for each & every possible word would be terribly slow. If the target vocabulary contains, say 50,000 french words, then the decoder would output 50,000-dimensional vectors, & then computing the softmax function over such a large vector would be very computationally intensive. To avoid this, one solution is to look only at the logits output by the model for the correct word & for a random sample of incorrect words, then compute an approximation of the loss based only on these logits. This *sampled softmax* technique was introduced in 2015 by Sebastien Jean et al. In TensorFlow, you can use the `tf.nn.sampled_softmax_loss()` function for this during training & use the normal softmax function during inference time (sampled softmax cannot be used at inference time because it required knowing the target).

The TensorFlow addons project includes many sequence-to-sequence tools to let you easily build production-ready encoder-decoders. For example, the following code creates a basic encoder-decoder model similar to the simple machine translation model from before.

In [ ]:
import tensorflow_addons as tfa

encoder_inputs = keras.layers.InputLayer(shape = [None], dtype = np.int32)
decoder_inputs = keras.layers.InputLayer(shape = [None], dtype = np.int32)
sequence_lengths = keras.layers.InputLayer(shape = [], dtype = np.int32)

embeddings = keras.layers.Embedding(vocab_size, embed_size)
encoder_embeddings = embeddings(encoder_inputs)
decoder_embeddings = embeddings(decoder_inputs)

encoder = keras.layers.LSTM(512, return_state = True)
encoder_outputs, state_h, state_c = encoder(encoder_embeddings)
encoder_state = [state_h, state_c]

sampler = tfa.seq2seq.sampler.TrainingSampler()

decoder_cell = keras.layers.LSTMCell(512)
output_layer = keras.layers.Dense(vocab_size)
decoder = tfa.seq2seq.basic_decoder.BasicDecoder(decoder_cell, sampler, 
                                                 output_layer = output_layer)
final_outputs, final_state, final_sequence_lengths = decoder(decoder_embeddings,
                                                             initial_state = encoder_state,
                                                             sequence_length = sequence_lengths)
Y_proba = tf.nn.softmax(final_outputs.rnn_output)

model = keras.Model(inputs = [encoder_inputs, decoder_inputs, sequence_lengths],
                    outputs = [Y_proba])

The code is mostly self-explanatory, but there are a few points to note. First, we set `return_state = True` when creating the `LSTM` layer so that we can get its final hidden state & pass it to the decoder. Since we are using an LSTM cell, it actually returns two hidden state (short term & long term). The `TrainingSampler` is one of several samplers available in TensorFlow addons: their role is to tell the decoder at each step what it should pretend the previous output was. During inference, this should be the embedding of the token that was actually output. During training, it should be the embedding of the previous target token: this is why we used the `TrainingSampler`. In practice, it is often a good idea to start training with the embedding of the target of the previous time step & gradually transition to using the embedding of the actual token that was output at the previous step. This idea was introduced in a 2015 paper by Samy Bengio et al. The *ScheduleEmbeddingTrainingSampler` will randomly choose between teh target or the actual output, with a probability that you can gradually change during training.

## Bidirectional RNNs

At each time step, a regular recurrent layer only looks at past & present inputs before generating its output. In other words, it is "causal", meaning it cannot look into the future. This type of RNN makes sense when forecasting time series, but for many NLP tasks, such as neural machine translation, it is often preferable to look ahead at the next words before encoding a given word. For example, consider the phrases "the queen of the United Kingdom", "the queen of hearts" & "the queen bee": to properly encode the word "queen", you need to look ahead. To implement this, run two recurrent layers on the same inputs, one reading the words from left to right & the other reading them from right to left. Then simple combine their outputs at each time step, typically by concatenating them. This is called a *bidirectional recurrent layer*.

<img src = "Images/Bidirectional Recurrent Layer.png" width = "400" style = "margin:auto"/>

To implement a bidirectional recurrent layer in keras, wrap a recurrent layer in a `keras.layers.Bidirectional` layer. For example, the following code creates a bidirectional `GRU` layer:

In [ ]:
keras.layers.Bidirectional(keras.layers.GRU(10, return_sequence = True))

## Beam Search

Suppose you train an encoder-decoder model & use it to transalte the french sentence "Comment vas-tu?" to english. You are hoping that it will output the proper translation ("How are you?"), but unfortunately, it outputs "How will you?" Looking at the training set, you notice many sentences such as "Comment vas-tu jouer?" which translates to "How will you play?" So it wasn't absurd for the model to output "How will" after seeing "Comment vas". Unfortunately, in this case it was a mistake & the model could not go back & fix it, so it tried to complete the sentence as best it could. By greedily outputting the most likely word at every step, it ended up with a suboptimal translation. How can we give the model a chance to go back & fix mistakes it made earlier? One of the most common solutions is *beam search*: it keeps track of a short list of the *k* most promising sentences (say, the top three), & at each decoder step it tries to extend them by one word, keeping only the *k* most likely sentences. The parameter *k* is called the *beam width*.

For example, suppose you use the model to translate the sentence "Comment vas-tu?" using beam search with a beam width of 3. At the first decoder step, the model will output an estimated probability for each possible word. Suppose the top three words are "How" (75% estimated probability), "What" (3%), & "You" (1%). That's our short list so far. Next, we create three copies of our model & use them to find the next word for each sentence. Each model will output one estimated probability per word in the vocabulary. The first model will try to find the next word in the sentence "How", & perhaps it will output a probability og 36% for the word "will", 32% forthe word "are", 16% for the word "do", & so on. Note that these are actually *conditional* probabilities, given that the sentence starts with "How". The second model will try to complete the sentence "What"; it might output a conditional probability of 50% for the word "are", & so on. Assuming the vocabulary has 10,000 words, each model will output 10,000 probabilities.

Next, we compute the probabilities of each of the 30,000 two-wrod sentences that these models considered (3 x 10,000). We do this by multiplying the estimated conditional probability of each word by the estimated probability of the sentence it completes. For example, the estimated probability of the sentence "How" was 75%, while the estimated conditional probability of the word "will" (given that the first word is "How") was 36%, so the estimated probability of the sentence "How will" is 75% x 36% = 27%. After computing the probabilities of all 30,000 two-word sentences, we keep only the top 3. Perhaps they all start with the word "How": "How will" (27%), "How are" (24%), & "How do" (12%). Right now, the sentence "How will" is winning, but "How are" has not been eliminated.

Then we repeat the same process: we use three models to predict the next word in each of these three sentences, & we compute the probabilities of all 30,000 three-word sentences we considered. Perhaps the top three are now "How are you" (10%), "How do you" (8%), & "How will you" (2%). At the next step we may get "How do you do" (7%), "How are you \<eos>" (6%), & "How are you doing" (3%). Notice that "How will" was eliminated, & we now have three perfectly reasonable translations. We boosted our encoder-decoder model's performance without any extra training, simply by using it more wisely.

You can implement beam search fairly easily using TensorFlow addons:

In [ ]:
beam_width = 10
decoder = tfa.seq2seq.beam_search_decoder.BeamSearchDecoder(cell = decoder_cell, 
                                                            beam_width = beam_width, 
                                                            output_layer = output_layer)
decoder_initial_state = tfa.seq2seq.beam_search_decoder.tile_batch(encoder_state,
                                                                   multiplier = beam_width)
outputs, _, _ = decoder(embedding decoder, start_tokens = start_tokens, end_token_end_token,
                        initial_state = decoder_initial_state)

We first create a `BeamSearchDecoder`, which wraps all the decoder clones (in this case 10 clones). Then we create one copy of the encoder's final state for each decoder clone, & we pass these states to the decoder, along with the start & end tokens.

With all this, you can get good translations for fairly short sentences (especially if you use pretrained word embeddings). Unfortunately, this model will be really bad at translating long sentences. Once again, the problem comes from the limited short term memory of RNNs. *Attention mechanisms* are the game-changing innovation that addressed this problem.

---

# Attention Mechanisms

Consider the path from the word "milk" to its translation "lait" in the simple machine translation model: it is quite long! This means that a representation of this word (along with all the other words) needs to be carried over many steps before it is actually used. Can't we make this path shorter?  This was the core idea in a groundbreaking 2014 paper by dzmitry bahdanau et al. They introduced a technique that allowed the decoder to focus on the appropriate words (as encoded by the encoder) at each time step. For example, at the time step where the decoder needs to output the word "lait", it will focus its attention on the word "milk". This means that the path from an input word to its translation is not much shorter, so the short-term memory limitations of RNNs have much less impact. Attention mechanisms revolutionalised neural machine trnaslation (& NLP in general), allowing a significant improvement in the state of the art, especially for long sentences (over 30 words).

The figure below shows this model's architecture (slighly simplified). 

<img src = "Images/NMT using Encoder-Decoder with Attention Model.png" width = "550" style = "margin:auto"/>

On the left, you have the encoder & the decoder. Instead of just sending the encoder's final hidden state to the decoder (which is still done, although it is not shown in the figure), we now send all of its outputs to the decoder. At each time step, the decoder's memory cell computes a weighted sum of all these encoder outputs: this determines which words it will focus on at this step. The weight $\alpha_{(t, i)}$ is the weight of the $i^{th}$ encoder output at the $t^{th}$ decoder time step. For example if the weight $\alpha_{(3, 2)}$ is much larger than the weights $\alpha_{(3, 0)}$ & $\alpha_{(3, 1)}$, then the decoder will pay much more attention to word number 2 ("milk") than to the other two words, at least at this time step. The rest of the decoder works just like earlier: at each time step, the memory cell receives the inputs we just discussed, plus the hidden state from the previous time step, & finally (although it is not represented in the diagram) it receives the target word from the previous time step (or at inference time, the output from the previous time step).

But where do these $\alpha_{(t, i)}$ weights come from? It's actually pretty simple: they are generated by a type of small neural network called an *alignment model* (or an *attention layer*), which is trained jointly with the rest of the encoder-decoder model. This alignment model is illustrated on the righthand side of the above figure. It starts with a time-distributed `Dense` later with a single neuron, which receives as input all the encoder outputs, concatenated with the decoder's previous hidden state (e.g., $h_{(2)}$). This layer outputs a score (or energy) for each encoder output (e.g., $e_{(3, 2)}$): this score measures how well each output is aligned with the decoder's previous hidden state. Finally, all the scores go through a softmax layer to get a final weight for each encoder output (e.g., $\alpha_{(3, 2)}$). All weights for a given decoder time step add up to 1 (since the softmax layer is not time-distributed). This particular attention mechanism is called *Bahdanau attention* (named after the paper's first author). Since it concatenates the encoder output with the decoder's previous hidden state, it is sometimes called *concatenative attention* (or *additive attention*).

Another common attention mechanism was proposed shortly after, in a 2015 paper by Minh-Thang Luong et al. Because the goal of the attention mechanism is to measure the similarity between one of the encoder's outputs & the decoder's previous hidden state, the authors proposed to simply compute the *dot product* of these two vectors, as this is often a fairly good similarity measure, & modern hardware can compute it much faster. For this to be possible, both vectors must have the same dimensionality. This is called *Luong attention* (again, after the paper's first author), or sometimes called *multiplicative attention*. The dot product gives a score, & all the scores (at a given decoder time step) go through a softmax layer to give the final weights, just like Bahdanau attention. Another simplification they proposed was to use the decoder's hidden state at the current time step rather than at the previous time step (i.e.,, $h_{(t)}$ rather than $h_{(t - 1)}$), then to use the output of the attention mechanism (noted $\tilde{h}_{(t)}$) directly to compute the decoder's predictions (rather than using it to compute the decoder's current hidden state). They also proposed a variant of the dot product mechanism where the encoder outputs first go through a linear transformation (i.e., a time-distributed `Dense` layer without a bias term) before the dot products are computed. This is called the "general" dot product approach. They compared both dot product approaches to the concatenative atttention mechanism (adding a rescaling parameter $v$), & they observed that the dot product variants performed better than concatenative attention. For this reason, concatenative attention is much less used now. The equations for these three attention mechanisms are summarise below:

$$\begin{split}
\tilde{h}_{(t)} = \sum_{i} \alpha_{(t, i)} y_{(i)} \\
with \quad \alpha_{(t, i)} = \frac{exp(e_{(t, i)})}{\sum_{i'} exp(e_{(t, i')})} \\
and \quad e_{(t, i)} = \Biggl\{ \begin{split}
h^{\intercal}_{(t)} y_{(i)} \quad \quad \quad dot\\
h^{\intercal}_{(t)} W y_{(i)} \quad general\\
v^{\intercal} tanh(W[h_{(t)}; y_{(i)}]) \quad concat
\end{split}
\end{split}$$

Here is how you can add Luong attention to an encoder-decoder model using TensorFlow addons:

In [ ]:
attention_mechanism = tfa.seq2seq.attention_wrapper.LuongAttention(units, 
                                                                   encoder_state,
                                                                   memory_sequence_length = encoder_sequence_length)
attention_decoder_cell = tfa.seq2seq.attention_wrapper.AttentionWrapper(decoder_cell,
                                                                        attention_mechanism,
                                                                        attention_layer_size = n_units)

We simply wrap the decoder cell in an `AttentionWrapper`, & we provide the desired attention mechanism (Luong attention in this example).

## Visual Attention

Attention mechanisms are now used for a variety of purposes. One of their first applications beyond NMT was in generatin gimage cations using visual attention: a convolutional neural network first processes the image & outputs some feature maps, then a decoder RNN equipped with an attention mechanism generates the caption, one word at a time. At each decoder time step (each word), the decoder uses the attention model to focus on just the right part of the image. For example in the below figure, the model geenrates the caption "A woman is throwing a frisbee in a park", & you can see what part of the input image the decoder focused its attention on when it was about to output the word "frisbee": clearly, most of its attention was focused on the frisbee.

<img src = "Images/Visual Attention.png" width = "600" style = "margin:auto"/>

Attention mechanisms are so powerful that you can actually build state-of-the art models using only attention mechanisms.

## Attention Is All You Need: The Transformer Architecture

In a groundbreaking 2017 paper, a team of google researcher suggested that "Attention Is All You Need". They managed to create an architecture called the *Transformer*, which significantly improved the state of the art in NMT without using any recurrent or convolutional layers, just attention mechanisms (plus embedding layers, dense layers, normalisation layers, & a few other bits & pieces). As an extra bonus, this architecture was also much faster to train & easier to parallelise, so they managed to train it at a fraction of the time & cost of the previous state-of-the-art models.

The transformer architecture is represented.

<img src = "Images/Transformer Architecture.png" width = "550" style = "margin:auto"/>

Let's walk through this figure:

* The lefthand part is the encoder. Just like earlier, it takes as input a batch of sentences represented as sequences of word IDs (the input shape is [*batch size*, *max input sentence length*]), & it encodes each word into a 512-dimensional representation (so the encoder's output shape is [*batch size*, *max input sentence length*, 512]). Note that the top part of the encoder is stacked N times (in the paper, N = 6).
* The righthand part is the decoder. During training, it takes the target sentence as input (also represented as a sequence of word IDs), shifted one time step to the right (i.e., a start-of-sequence token is inserted at the beginning). It also receives the outputs of the encoder (i.e., the arrows coming from the left side). Note that the top part of the decoder is also stacked N times, & the encoder's stack final outputs are fed to the decoder at each of these N levels. Just like earlier, the decoder outputs a probability for each possible next word, at each time step (its output shape is [*batch size*, *max output sentence length*, *vocabulary length*]).
* During inference, the decoder cannot be fed targets, so we feed it the previously output words (starting with a start-of-sequence token). So the model needs to be called repeatedly, predicting one more word at every round (which is fed to the decoder at the next round, until the end-of-sequence token is output).
* Looking more closely, you can see that you are already familiar with most components: there are two embedding layers, 5 * N skip connections, each of them followed by a layer normalisation layer, 2 * N "Feed Forward" modules that are composed of two dense layers each (the first one using the ReLU activation function, the second with no activation function), & finally the output layer is a dense layer using the softmax activation function. All of these layers are time-distributed, so each word is treated independently of all the other. But how can we translated a sentence by only looking at one word at a time? Well, that's where the new componenets come in:
   - The encoder's *Multi-Head Attention* layer encodes each word's relationship with every other word in the same sentence, paying more attention to the most relevant ones. For example, the output of this layer for the word "Queen" in the sentence "They welcomed the Queen of the United Kingdom" will depend on all the words in the sentence, but it will probably pay more attention to the words "United" & "Kingdon" than to the words "They" or "welcomed". This attention mechanism is called *self-attention* (the sentence is paying attention to itself). We will discuss exactly how it works shortly. The decoder's *Masked Mult-Head Attention* layer does the same thing, but each word is only allowed to attend to words located before it. Finally, the decoder's upper multi-head attention layer is where the decoder pays attention to the words in the input sentence. For example, the decoder will probably pay close attention to the word "Queen" in the input sentence when it is about to output this word's translation.
   - The *positional embeddings* are simply dense vectors (much like word embeddings) that represent the position of a word in the sentence. This gives the model access to each word's position, which is needed because the multi-head attention layers do not consider the order or the position of the words; they only look at their relationships. Since all the other layers are time-distributed, they have no way of knowing the position of each word (either relative or absolute). Obviously, the relative & absolute word positions are important, so we need to give this information to the transformer somehow, & positional embeddings are a good way to do this.
 
Let's look a bit closer at both these novel components of the transformer architecture, starting with positional embeddings.

### Positional Embeddings

A positional embedding is a dense vector that encodes the position of a word within a sentence: the $i^{th}$ positional embedding is simply added to the word embedding of the $i^{th}$ word in teh sentence. These positional embeddings can be learned by the model, but in the paper, the authors preferred to use fixed positional embeddings, defined using the sine & cosine functions of different frequencies. The positional embedding matrix $P$ is defined in the below equation, represented in the below figure, where $P_{p, i}$ is the $i^{th}$ component of the embedding for the word located at the $p^{th}$ position in the sentence.

$$\begin{split}
P_{p, 2i} = sin(p/10000^{2i/d}) \\
P_{p, 2i + 1} = cos(p/10000^{2i/d})
\end{split}$$

<img src = "Images/Positional Embedding Matrix.png" width = "600" style = "margin:auto"/>

This solution gives the same performance as learned positional embeddings do, but it can extend to arbitrarily long sentences, which is why it's favoured. After the positional embeddings are added to the word embeddings, the rest of the model has access to the absolute position of each word in the sentence because there is a unique positional embedding for each position (e.g., the positional embedding for the word located at the 22nd position in a sentence is represented by the vertical dashed line at the bottom of the above figure, & you can see that it is unique to that position). Moreover, the choice of oscillating function (sine & cosine) makes it possible for the model to learn relative positions as well. For example, words located 38 words apart (e.g., at position p = 22 & p = 60) always have the same positional embedding values in the embedding dimensions i = 100 & i = 101, as you can see in the above figure. This explains why we need both the sine & cosine for each frequency: if we only use sine (the blue wave at i = 100), the model would not be able to distinguish positions p = 25 & p = 35 (marked by a cross).

There is no `PositionalEmbedding` layer in TensorFlow, but it easy to create on. For efficiency reasons, we precompute the positional embedding matrix in the constructor (so we need to know the maximum sentence length, `max_steps`, & the number of dimensions for each word representation, `max_dims`). Then the `call()` method crops this embedding matrix to the size of the inputs, & it adds it to the inputs. Since we added an extra first dimension of size 1 when creating the positional embedding matrix, the rules of broadcasting will ensure that the matrix gets added to every sentence in the inputs:

In [ ]:
class PositionalEncoding(keras.layers.Layer):
    def __init__(self, max_steps, max_dims, dtype = tf.float32, **kwargs):
        super().__init__(dtype = dtype, **kwargs):
        if max_dims % 2 == 1: 
            max_dims += 1
        p, i = np.meshgrid(np.arange(max_steps), np.arange(max_dims // 2))
        pos_emb = np.empty((1, max_steps, max_dims))
        pos_emb[0, :, ::2] = np.sin(p / 10000**(2 * i / max_dims)).T
        pos_emb[0, :, 1::2] = np.cos(p / 10000**(2 * i / max_dims)).T
        self.positional_embedding = tf.constant(pos_emb.astype(self.dtype))
    def call(self, inputs):
        shape = tf.shape(inputs)
        return inputs + self.positional_embedding[:, :shape[-2], :shape[-1]]

Then we can create the first layers of the transformer:

In [ ]:
embed_size = 512; max_steps = 500; vocab_size = 10000
encoder_inputs = keras.layers.Input(shape = [None], dtype = np.int32)
decoder_inputs = keras.layers.Input(shape = [None], dtype = np.int32)
embeddings = keras.layers.Embedding(vocab_size, embed_size)
encoder_embeddings = embeddings(encoder_inputs)
decoder_embeddings = embeddings(decoder_inputs)
positional_encoding = PositionalEncoding(max_steps, max_dims = embed_size)
encoder_in = positional_encoding(encoder_embeddings)
decoder_in = positional_encoding(decoder_embeddings)

Now, let's look deeper into the heart of the transformer model: the multi-head attention layer.

### Multi-Head Attention

To understand how a multi-head attention layer works, we must first understand the *scaled dot-product attention* layer, which it is based on. Let's suppose the encoder analysed the input sentence, "They played chess", & it managed to understand that the word "They" is the subject & the word "played" is the verb, so it encoded this information in the representations of these words. Now suppose the decoder has already translated the subject, & it thinks that it should translate the verb next. For this, it needs to fetch the verb from the input sentence. This is analog to a dictionary lookup: it's as if the encoder created a dictionary {"subject", "They", "verb", "played", ...} & the decoder wanted to look up the value that corresponds to the key "verb". However, the model does not have discrete tokens to represent the keys (like "subject" or "verb"); it has vectorized representations of these concepts (which it learned during training), so the key it will use for the lookup (called the *query*) will not perfectly match any key in the dictionary. The solution is to compute a similarity measure between the query & each key in the dictionary, & then use a softmax function to convert these similarity scores to weights that add up to 1. If the key that represents the verb is by far the most similar to the query, then the weighted sum will be very close to the representation of the word "played". In short, you can think of this whole process as a differentiable dictionary lookup. The similarity measure used by the transformer is just the dot product, like in Luong attention. In fact, the equation is the same as for Luong attention, except for a scaling factor. The equation is shows below in vectorised form.

$$Attention(Q, K, V) = softmax(\frac{QK^{\intercal}}{\sqrt{d_{keys}}})V$$

In this equation:

* $Q$ is a matrix containing one row per query. Its shape is [$n_{queries}$, $d_{keys}$], where $n_{queries}$ is the number of queries & $d_{keys}$ is the number of dimensions of each query & each key.
* $K$ is a matrix containing one row per key. Its shape is [$n_{keys}$, $d_{keys}$], where $n_{keys}$ is the number of keys & values.
* $V$ is a matrix containing one row per value. Its shape is [$n_{keys}$, $d_{values}$] where $d_{values}$ is the number of each value.
* The shape of $QK^{\intercal}$ is [$n_{queries}$, $n_{keys}$]: it contains one similarity score for each query/key pair. The output of the softmax function has the same shape, but all rows sum up to 1. The final output has a shape of [$n_{queries}$, $d_{values}$]: there is one row per query, where each row represents the query result (a weighted sum of the values).
* The scaling factor scales down the similarity scores to avoid saturating the softmax function, which would lead to tiny gradients.
* It is possible to mask out some key/value pairs by adding a very large negative value to the corresponding similarity scores, just before computing the softmax. This is useful in the masked multi-head attention layer.

In the encoder, this equation is applied to every input sentence in the batch, with Q, K, & V all equal to the list of words in the input sentence (so each word in the sentence will be compared to every word in the same sentence, including itself). Similarly, in the decoder's masked attention layer, the equation will be applied to every target sentence in the batch, with $Q$, $K$, & $V$ all equal to the list of words in the target sentence, but this time using a mask to prevent any word from comparing itself words located after it (at inference time the decoder, the keys $K$ & values $V$ are simply the list of word encodings produced by the encoder, & the queries $Q$ are the list of word encodings produced by the decoder.

The `keras.layers.Attention` layer implements scaled dot-product attention, efficiently applying the above equation to multiple sentences in a batch. Its inputs are just like $Q$, $K$, & $V$, except with an extra batch dimension (the first dimension). 

If we ignore the skip connections, the layer normalisation layers, the feed forward blocks & the fact that this is a sclaed dot-product attention, not exactly multi-head attention, then the rest of the transformer model can be implemented like this:

In [ ]:
Z = encoder_in
for N in range(6):
    Z = keras.layers.Attention(use_scale = True)([Z, Z])

encoder_outputs = Z
Z = decoder_in
for N in range(6):
    Z = keras.layers.Attention(use_scale = True, causal = True)([Z, Z])
    Z = keras.layers.Attention(use_sclae = True)([Z, encoder_outputs])

outputs = keras.layers.TimeDistributed(keras.layers.Dense(vocab_size,
                                                          activation = "softmax"))(Z)

The `use_scale = True` argument creates an additional parameter that lets the layer learn how to properly downscale the similarity scores. This is a bit different from the transformer model, which always downscales the similarity scores by the same factor ($\sqrt{d_{keys}}$). The `causal = True` argument when creating the second attention layer ensures that each output token only attends to previous output tokens, not future ones.

Now it's time to look at the final piece of the puzzle: what is a multi-head attention layer? Its architecture is shown below:

<img src = "Images/Multi-Head Attention Layer Architecture.png" width = "500" style = "margin:auto"/>

As you can see, it is just a bunch of scaled dot-product attention layers, each preceded by a linear transformation of the values, keys & queries (i.e., a time-distributed `Dense` layer with no activation function). All the outputs are simply concatenated, & they go through a final linear transformation (again, time-distributed). But why? What is the intuition behind this architecture? Well, consider the word "played" we discussed earlier (in the sentence "They played chess"). The encoder was smart enough to encode the fact that it is a verb. But the word representation also includes many other feature that are useful for its translation, such as the fact that it is in the past tense. In short, the word representation encodes many different characteristics of the word. if we just used a single scaled dot-product attention layer, we would only be able to query all of these characteristics in one shot. This is why the multi-head attention layer applies multiple different linear transformations of the value, keys, & queries: this allows the model to apply many different projections of the word representation into different subspaces, each focusing on a subset of the word's characteristics. Perhaps one of the linear layers will project the word representation into a subspace where all that remains is the information that the word is a verb, another linear layer will extract just the fact that is past tense, & so on. Then the scaled dot-product attention layers implement the lookup phase, & finally we concatenate all the results & project them back to the original space.

At the this current time, you can check out TensorFlow's tutorial for building a transformer model for language understanding. Moreover the tf hub team is currently porting several transformer-based modules to TensorFlow 2, & they should be available by now.

---

# Recent Innovations in Language Models

The year 2018 has been called the "imagenet moment for NLP": progress was astounding, with larger & alrger LSTM & transformer-based architectures trained on immense datasets. i highly recommend you check out the following papers, all published in 2018:

* The ELMo paper by Matthew Peters introduced *Embeddings from Language Models* (ELMo): these are contextualised word embeddings learned from the internal states of a deep bidirectional language model. For example, the word "queen" will not have the same embedding in "Queen of the United Kingdom" & in  "queen bee".
* The ULMFiT paper by Jeremy Howard & Sebastian Ruder demonstrated the effectiveness of unsupervised pretraining for NLP tasks: the authors trained a LSTM language model using self-supervised learning (i.e., generating the labels automatically from the data) on a huge text corpus, then they fine-tuned it on various tasks. Their model outperformed the state of the art on six text classification tasks by a large margin (reducing the error rate by 18-24% in most cases). Moreover, they showed that by fine-tuning the pretrained model on just 100 labeled examples, they could achieve the same performance as a model trained from scratch on 10,000 examples.
* The GPT paper by Alec Radford & other OpenAI researchers also demonstrated the effectiveness of unsupervised pretraining, but this time using a transformer-like architecture. The authors pretrained a large but fairly simple architecture composed of a stack of 12 transformer modules (using only masked multi-head attention layers) on a large dataset, once again trained using self-supervised learning. Then they fine-tuned it on various language tasks, using only minor adaptations for each task. The tasks were quite diverse: they included text classification, *entailment* (whether sentence A entails sentence B), similarity (e.g., "Nice weather today" is very similar to "It is sunny"), & question answering (given a few multiple-choice questions). Just a few months layer, in February 2019, Alec Radford, Jeffrey Wu, & other OpenAI researchers published the GPT-2 paper, which proosed a very similar architecture, but larger still (with over 1.5 billion parameters!) & they showed that it could achieve good performance on many tasks without any fine-tuning. This is called *zero-shot learning* (ZSL). A smaller version of GPT-2 model (with just 115 million parameters) is available at *https://github.openai/gpt-2*, along with its pretrained weights.
* The BERT paper by Jacob Devlin & other google researchers also demonstrates the effectiveness of self-supervised pretraining on a large corpus, using a similar architecture to GPT but non-masked multi-head attention layers (like in the transformer's encoder). This means that the model is naturally bidirectianl; hence the B in Bert (*Bidirectional Encoder Representations from Transformers*). Most importantly, the authors proposed two pretraining tasks that explain most of the model's strength:
   - *Masked language model* (MLM): each word in a sentence has a 15% probability of being masked, & the model is trained to predict the masked words. For example, if the original sentence is "She had fun at the birthday party", then the model may be given the sentence "She \<mask> fun at the \<mask> party", & it must predict the words "had" & "birthday" (the other outputs will be ignored). To be more precise, each selected word has an 80% chance of being masked, a 10% chance of being replaced by a random word (to reduce the discrepancy between pretraining & fine-tuning, since the model will not see \<mask> tokens during fine-tuning), & a 10% chance of beling left alone (to bias the model toward the correct answer).
   - *Next sentence prediction* (NSP): The model is trained to predict whether two sentences are consecutive or not. For example, it should predict that "The dog sleeps" & "It snores loudly" are consecutive sentences, while "The dog sleeps" & "The Earth orbits the Sun" are not consecutive. This is a challenging task, & it significantly improved the performance of the model when it is fine-tuned on tasks such as question answering or entailment.
 
As you can see, the main innovations in 2018 & 2019 have been better subword tokenisation, shifting from LSTMs to Transformers, & pretraining universal language models using self-supervised learning, then fine-tuning them with very few architectural changes (or none at all). Things are moving fast: no one can say what architectures will prevail next year. Today, it's clearly transformers, but tomorrow it might be CNNs (e.g., check out the 2018 paper by Maha Elbayad et al., where the researchers used masked 2D convolutional layers for sequence-to-sequence tasks). Or it might even be RNNs, if htey make a surprise comeback (e.g., check out the 2018 paper by Shuai Li et al. that shows that by making neurons independent of each other in a given RNN layer, it is possible to train much deeper RNNs capable of learning much longer sequences).